todo
- get labeling files
- extract samples with agreement
- compute features for those
- run model predictions
- retrurn performance

In [1]:
import bento
import numpy as np
import pandas as pd
# from hmr import Reloader

# bento = Reloader(bento)

In [2]:
import matplotlib.pyplot as plt

In [3]:
consensus_data = pd.read_csv(
    "../../data/multilabel_annotations/consensus_labels_600.csv", index_col=0
).reset_index()

In [4]:
query = consensus_data.loc[consensus_data["dataset"] == "seqfish"]
query2 = consensus_data.loc[consensus_data["dataset"] == "merfish"]

In [5]:
seqfish = bento.io.read_h5ad("../../data/multilabel_annotations/seqfish_annot.h5ad")

seqfish_genes = np.unique(query["gene"]).tolist()
seqfish_cells = np.unique(query["cell"]).tolist()
seqfish = seqfish[seqfish_cells, seqfish_genes]
bento.pp.set_points(seqfish)


AttributeError: module 'bento' has no attribute 'pp'

In [6]:
merfish = bento.io.read_h5ad("../../data/multilabel_annotations/u2os_merfish_annot.h5ad")
merfish_genes = np.unique(query2["gene"]).tolist()
merfish_cells = np.unique(query2["cell"]).tolist()
merfish = merfish[merfish_cells, merfish_genes]
bento.pp.set_points(merfish)

Trying to set attribute `.uns` of view, copying.


In [7]:
patterns = consensus_data.columns.tolist()[-5:]
patterns

['cell_edge', 'cytoplasmic', 'none', 'nuclear', 'nuclear_edge']

In [8]:
for p in patterns:
    seqfish.layers[p] = consensus_data.pivot_table(
        index="cell", columns="gene", values=p
    ).loc[seqfish_cells, seqfish_genes]
    merfish.layers[p] = consensus_data.pivot_table(
        index="cell", columns="gene", values=p
    ).loc[merfish_cells, merfish_genes]

In [9]:
consensus_data["pattern"] = (
    consensus_data[patterns]
    .apply(lambda col: col.replace({1: col.name, 0: ""}))
    .apply(lambda row: "".join(row.values), axis=1)
)

In [ ]:
def plot_temp(row):
    fig = bento.pl.plot_cells(
        merfish, cells=[row["cell"]], genes=[row["gene"]], cmap="blue", size=2
    )
    plt.suptitle(row["pattern"])


consensus_data.loc[consensus_data["dataset"] == "merfish"].apply(
    lambda row: plot_temp(row), axis=1
)

In [10]:
def compute_features(data):
    feature_names = []

    bento.tl.ShapeProximity("cell_shape").transform(data)
    bento.tl.ShapeProximity("nucleus_shape").transform(data)
    bento.tl.ShapeAsymmetry("cell_shape").transform(data)
    bento.tl.ShapeAsymmetry("nucleus_shape").transform(data)
    bento.tl.Ripley().transform(data)
    bento.tl.PointDispersion().transform(data)
    bento.tl.ShapeDispersion("nucleus_shape").transform(data)

In [12]:
# bento.reload()
compute_features(merfish)

AnnData object modified:
    obs:
        + cell_radius
[########################################] | 100% Completed | 14.0s
AnnData object modified:
    obs:
        + cell_radius
    layers:
        + cell_outer_proximity, cell_inner_proximity
[########################################] | 100% Completed | 10.7s
AnnData object modified:
    layers:
        + nucleus_inner_proximity, nucleus_outer_proximity
[########################################] | 100% Completed | 13.4s
AnnData object modified:
    layers:
        + cell_inner_asymmetry, cell_outer_asymmetry
[########################################] | 100% Completed | 10.1s
AnnData object modified:
    layers:
        + nucleus_inner_asymmetry, nucleus_outer_asymmetry
[########################################] | 100% Completed | 13.9s
AnnData object modified:
    layers:
        + l_min_gradient, l_monotony, l_max_gradient, l_half_radius, l_max
[########################################] | 100% Completed |  3.6s
AnnData object modifi

In [13]:
# bento.reload()
compute_features(seqfish)

AnnData object modified:
    obs:
        + cell_radius
[########################################] | 100% Completed | 10.5s
AnnData object modified:
    obs:
        + cell_radius
    layers:
        + cell_outer_proximity, cell_inner_proximity
[########################################] | 100% Completed |  8.0s
AnnData object modified:
    layers:
        + nucleus_inner_proximity, nucleus_outer_proximity
[########################################] | 100% Completed |  9.6s
AnnData object modified:
    layers:
        + cell_inner_asymmetry, cell_outer_asymmetry
[########################################] | 100% Completed |  7.6s
AnnData object modified:
    layers:
        + nucleus_inner_asymmetry, nucleus_outer_asymmetry
[########################################] | 100% Completed | 50.8s
AnnData object modified:
    layers:
        + l_min_gradient, l_monotony, l_max_gradient, l_half_radius, l_max
[########################################] | 100% Completed |  4.1s
AnnData object modifi

In [14]:
bento.io.write_h5ad(
    seqfish,
    "../../data/multilabel_annotations/seqfish_annot_gold.h5ad",
)
bento.io.write_h5ad(
    merfish,
    "../../data/multilabel_annotations/merfish_annot_gold.h5ad",
)